In [105]:
import os
from datetime import datetime, time
from datetime import timezone

import pandas as pd

In [106]:
file_path = r"..\data"
file_name = r"Annual Smart Meter Consumption.csv"
solar_data = pd.read_csv(os.path.join(file_path, file_name))
solar_data

,Consumption (kWh),Start,End
0,0.507,2021-09-29T00:00:00+01:00,2021-09-29T00:30:00+01:00
1,1.318,2021-09-29T00:30:00+01:00,2021-09-29T01:00:00+01:00
2,1.779,2021-09-29T01:00:00+01:00,2021-09-29T01:30:00+01:00
3,1.273,2021-09-29T01:30:00+01:00,2021-09-29T02:00:00+01:00
4,1.309,2021-09-29T02:00:00+01:00,2021-09-29T02:30:00+01:00
...,...,...,...
17517,0.769,2022-09-28T22:30:00+01:00,2022-09-28T23:00:00+01:00
17518,1.349,2022-09-28T23:00:00+01:00,2022-09-28T23:30:00+01:00
17519,1.388,2022-09-28T23:30:00+01:00,2022-09-29T00:00:00+01:00
17520,1.400,2022-09-29T00:00:00+01:00,2022-09-29T00:30:00+01:00


In [70]:
# Start Date Cleaning
def datetime_cleaner(data):
    """
    """
    remove_spaces = [time.replace(" ", "") for time in data[" Start"]]
    iso_format = [datetime.fromisoformat(time).astimezone(timezone.utc) for time in remove_spaces]
    time = [time.time() for time in iso_format]
    standard_format = [time.replace(tzinfo=None) for time in iso_format]
    data["iso_start"] = standard_format
    data["start_time"] = time
    
    remove_spaces = [time.replace(" ", "") for time in data[" End"]]
    iso_format = [datetime.fromisoformat(time).astimezone(timezone.utc) for time in remove_spaces]
    time = [time.time() for time in iso_format]
    standard_format = [time.replace(tzinfo=None) for time in iso_format]
    data["iso_end"] = standard_format
    data["end_time"] = time

    data["time_delta"] = data["iso_end"] - data["iso_start"]
    
    return data

clean_data = datetime_cleaner(solar_data)
clean_data

,Consumption (kWh),Start,End,iso_start,start_time,iso_end,end_time,time_delta
0,0.507,2021-09-29T00:00:00+01:00,2021-09-29T00:30:00+01:00,2021-09-28 23:00:00,23:00:00,2021-09-28 23:30:00,23:30:00,0 days 00:30:00
1,1.318,2021-09-29T00:30:00+01:00,2021-09-29T01:00:00+01:00,2021-09-28 23:30:00,23:30:00,2021-09-29 00:00:00,00:00:00,0 days 00:30:00
2,1.779,2021-09-29T01:00:00+01:00,2021-09-29T01:30:00+01:00,2021-09-29 00:00:00,00:00:00,2021-09-29 00:30:00,00:30:00,0 days 00:30:00
3,1.273,2021-09-29T01:30:00+01:00,2021-09-29T02:00:00+01:00,2021-09-29 00:30:00,00:30:00,2021-09-29 01:00:00,01:00:00,0 days 00:30:00
4,1.309,2021-09-29T02:00:00+01:00,2021-09-29T02:30:00+01:00,2021-09-29 01:00:00,01:00:00,2021-09-29 01:30:00,01:30:00,0 days 00:30:00
...,...,...,...,...,...,...,...,...
17517,0.769,2022-09-28T22:30:00+01:00,2022-09-28T23:00:00+01:00,2022-09-28 21:30:00,21:30:00,2022-09-28 22:00:00,22:00:00,0 days 00:30:00
17518,1.349,2022-09-28T23:00:00+01:00,2022-09-28T23:30:00+01:00,2022-09-28 22:00:00,22:00:00,2022-09-28 22:30:00,22:30:00,0 days 00:30:00
17519,1.388,2022-09-28T23:30:00+01:00,2022-09-29T00:00:00+01:00,2022-09-28 22:30:00,22:30:00,2022-09-28 23:00:00,23:00:00,0 days 00:30:00
17520,1.400,2022-09-29T00:00:00+01:00,2022-09-29T00:30:00+01:00,2022-09-28 23:00:00,23:00:00,2022-09-28 23:30:00,23:30:00,0 days 00:30:00


In [71]:
clean_data["Cost (pkWh)"] = 0

clean_data.loc[
    (clean_data["start_time"] >= time(5, 30))
    | (clean_data["start_time"] < time(23, 30)),
    "Cost (pkWh)"
] = 39.0

clean_data.loc[
    (clean_data["start_time"] >= time(23, 30))
    | (clean_data["start_time"] < time(5, 30)),
    "Cost (pkWh)"
] = 7.5

clean_data

,Consumption (kWh),Start,End,iso_start,start_time,iso_end,end_time,time_delta,Cost (pkWh)
0,0.507,2021-09-29T00:00:00+01:00,2021-09-29T00:30:00+01:00,2021-09-28 23:00:00,23:00:00,2021-09-28 23:30:00,23:30:00,0 days 00:30:00,39.0
1,1.318,2021-09-29T00:30:00+01:00,2021-09-29T01:00:00+01:00,2021-09-28 23:30:00,23:30:00,2021-09-29 00:00:00,00:00:00,0 days 00:30:00,7.5
2,1.779,2021-09-29T01:00:00+01:00,2021-09-29T01:30:00+01:00,2021-09-29 00:00:00,00:00:00,2021-09-29 00:30:00,00:30:00,0 days 00:30:00,7.5
3,1.273,2021-09-29T01:30:00+01:00,2021-09-29T02:00:00+01:00,2021-09-29 00:30:00,00:30:00,2021-09-29 01:00:00,01:00:00,0 days 00:30:00,7.5
4,1.309,2021-09-29T02:00:00+01:00,2021-09-29T02:30:00+01:00,2021-09-29 01:00:00,01:00:00,2021-09-29 01:30:00,01:30:00,0 days 00:30:00,7.5
...,...,...,...,...,...,...,...,...,...
17517,0.769,2022-09-28T22:30:00+01:00,2022-09-28T23:00:00+01:00,2022-09-28 21:30:00,21:30:00,2022-09-28 22:00:00,22:00:00,0 days 00:30:00,39.0
17518,1.349,2022-09-28T23:00:00+01:00,2022-09-28T23:30:00+01:00,2022-09-28 22:00:00,22:00:00,2022-09-28 22:30:00,22:30:00,0 days 00:30:00,39.0
17519,1.388,2022-09-28T23:30:00+01:00,2022-09-29T00:00:00+01:00,2022-09-28 22:30:00,22:30:00,2022-09-28 23:00:00,23:00:00,0 days 00:30:00,39.0
17520,1.400,2022-09-29T00:00:00+01:00,2022-09-29T00:30:00+01:00,2022-09-28 23:00:00,23:00:00,2022-09-28 23:30:00,23:30:00,0 days 00:30:00,39.0


In [72]:
clean_data["30 min cost"] = clean_data["Cost (pkWh)"] * clean_data["Consumption (kWh)"]
clean_data

,Consumption (kWh),Start,End,iso_start,start_time,iso_end,end_time,time_delta,Cost (pkWh),30 min cost
0,0.507,2021-09-29T00:00:00+01:00,2021-09-29T00:30:00+01:00,2021-09-28 23:00:00,23:00:00,2021-09-28 23:30:00,23:30:00,0 days 00:30:00,39.0,19.7730
1,1.318,2021-09-29T00:30:00+01:00,2021-09-29T01:00:00+01:00,2021-09-28 23:30:00,23:30:00,2021-09-29 00:00:00,00:00:00,0 days 00:30:00,7.5,9.8850
2,1.779,2021-09-29T01:00:00+01:00,2021-09-29T01:30:00+01:00,2021-09-29 00:00:00,00:00:00,2021-09-29 00:30:00,00:30:00,0 days 00:30:00,7.5,13.3425
3,1.273,2021-09-29T01:30:00+01:00,2021-09-29T02:00:00+01:00,2021-09-29 00:30:00,00:30:00,2021-09-29 01:00:00,01:00:00,0 days 00:30:00,7.5,9.5475
4,1.309,2021-09-29T02:00:00+01:00,2021-09-29T02:30:00+01:00,2021-09-29 01:00:00,01:00:00,2021-09-29 01:30:00,01:30:00,0 days 00:30:00,7.5,9.8175
...,...,...,...,...,...,...,...,...,...,...
17517,0.769,2022-09-28T22:30:00+01:00,2022-09-28T23:00:00+01:00,2022-09-28 21:30:00,21:30:00,2022-09-28 22:00:00,22:00:00,0 days 00:30:00,39.0,29.9910
17518,1.349,2022-09-28T23:00:00+01:00,2022-09-28T23:30:00+01:00,2022-09-28 22:00:00,22:00:00,2022-09-28 22:30:00,22:30:00,0 days 00:30:00,39.0,52.6110
17519,1.388,2022-09-28T23:30:00+01:00,2022-09-29T00:00:00+01:00,2022-09-28 22:30:00,22:30:00,2022-09-28 23:00:00,23:00:00,0 days 00:30:00,39.0,54.1320
17520,1.400,2022-09-29T00:00:00+01:00,2022-09-29T00:30:00+01:00,2022-09-28 23:00:00,23:00:00,2022-09-28 23:30:00,23:30:00,0 days 00:30:00,39.0,54.6000


In [73]:
columns = ["iso_start", "30 min cost"]
daily_cost = clean_data.groupby(clean_data["iso_start"].dt.date)[["30 min cost"]].sum()
daily_cost = daily_cost + 44.48
daily_cost

,30 min cost
iso_start,
2021-09-28,74.1380
2021-09-29,690.7970
2021-09-30,963.2090
2021-10-01,548.6765
2021-10-02,1130.4980
...,...
2022-09-24,554.1665
2022-09-25,1091.9300
2022-09-26,584.7650


In [74]:
clean_data

,Consumption (kWh),Start,End,iso_start,start_time,iso_end,end_time,time_delta,Cost (pkWh),30 min cost
0,0.507,2021-09-29T00:00:00+01:00,2021-09-29T00:30:00+01:00,2021-09-28 23:00:00,23:00:00,2021-09-28 23:30:00,23:30:00,0 days 00:30:00,39.0,19.7730
1,1.318,2021-09-29T00:30:00+01:00,2021-09-29T01:00:00+01:00,2021-09-28 23:30:00,23:30:00,2021-09-29 00:00:00,00:00:00,0 days 00:30:00,7.5,9.8850
2,1.779,2021-09-29T01:00:00+01:00,2021-09-29T01:30:00+01:00,2021-09-29 00:00:00,00:00:00,2021-09-29 00:30:00,00:30:00,0 days 00:30:00,7.5,13.3425
3,1.273,2021-09-29T01:30:00+01:00,2021-09-29T02:00:00+01:00,2021-09-29 00:30:00,00:30:00,2021-09-29 01:00:00,01:00:00,0 days 00:30:00,7.5,9.5475
4,1.309,2021-09-29T02:00:00+01:00,2021-09-29T02:30:00+01:00,2021-09-29 01:00:00,01:00:00,2021-09-29 01:30:00,01:30:00,0 days 00:30:00,7.5,9.8175
...,...,...,...,...,...,...,...,...,...,...
17517,0.769,2022-09-28T22:30:00+01:00,2022-09-28T23:00:00+01:00,2022-09-28 21:30:00,21:30:00,2022-09-28 22:00:00,22:00:00,0 days 00:30:00,39.0,29.9910
17518,1.349,2022-09-28T23:00:00+01:00,2022-09-28T23:30:00+01:00,2022-09-28 22:00:00,22:00:00,2022-09-28 22:30:00,22:30:00,0 days 00:30:00,39.0,52.6110
17519,1.388,2022-09-28T23:30:00+01:00,2022-09-29T00:00:00+01:00,2022-09-28 22:30:00,22:30:00,2022-09-28 23:00:00,23:00:00,0 days 00:30:00,39.0,54.1320
17520,1.400,2022-09-29T00:00:00+01:00,2022-09-29T00:30:00+01:00,2022-09-28 23:00:00,23:00:00,2022-09-28 23:30:00,23:30:00,0 days 00:30:00,39.0,54.6000


In [75]:
solar_output_per_kw = 800
solar_cost_per_kw = 1800
kw = 3.5
solar_output_year = kw*solar_output_per_kw
initial_outlay = solar_cost_per_kw*kw
avg_solar_output_30_mins = solar_output_year/17520

In [76]:
avg_solar_output_30_mins

0.1598173515981735

In [77]:
clean_data['Solar Output 30 mins (kWh)'] = avg_solar_output_30_mins

In [78]:
clean_data['Cumulative Solar Output (kWh)'] = clean_data['Solar Output 30 mins (kWh)'].cumsum()

In [79]:
clean_data['Cost Saved 30 mins (p)'] = clean_data['Cost (pkWh)']*clean_data['Solar Output 30 mins (kWh)']

In [80]:
clean_data['Cumulative Cost Saved (p)'] = clean_data['Cost Saved 30 mins (p)'].cumsum()

In [81]:
clean_data['Cumulative Non-Solar Cost (p)'] = clean_data['30 min cost'].cumsum()

In [82]:
clean_data

,Consumption (kWh),Start,End,iso_start,start_time,iso_end,end_time,time_delta,Cost (pkWh),30 min cost,Solar Output 30 mins (kWh),Cumulative Solar Output (kWh),Cost Saved 30 mins (p),Cumulative Cost Saved (p),Cumulative Non-Solar Cost (p)
0,0.507,2021-09-29T00:00:00+01:00,2021-09-29T00:30:00+01:00,2021-09-28 23:00:00,23:00:00,2021-09-28 23:30:00,23:30:00,0 days 00:30:00,39.0,19.7730,0.159817,0.159817,6.232877,6.232877,19.7730
1,1.318,2021-09-29T00:30:00+01:00,2021-09-29T01:00:00+01:00,2021-09-28 23:30:00,23:30:00,2021-09-29 00:00:00,00:00:00,0 days 00:30:00,7.5,9.8850,0.159817,0.319635,1.198630,7.431507,29.6580
2,1.779,2021-09-29T01:00:00+01:00,2021-09-29T01:30:00+01:00,2021-09-29 00:00:00,00:00:00,2021-09-29 00:30:00,00:30:00,0 days 00:30:00,7.5,13.3425,0.159817,0.479452,1.198630,8.630137,43.0005
3,1.273,2021-09-29T01:30:00+01:00,2021-09-29T02:00:00+01:00,2021-09-29 00:30:00,00:30:00,2021-09-29 01:00:00,01:00:00,0 days 00:30:00,7.5,9.5475,0.159817,0.639269,1.198630,9.828767,52.5480
4,1.309,2021-09-29T02:00:00+01:00,2021-09-29T02:30:00+01:00,2021-09-29 01:00:00,01:00:00,2021-09-29 01:30:00,01:30:00,0 days 00:30:00,7.5,9.8175,0.159817,0.799087,1.198630,11.027397,62.3655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17517,0.769,2022-09-28T22:30:00+01:00,2022-09-28T23:00:00+01:00,2022-09-28 21:30:00,21:30:00,2022-09-28 22:00:00,22:00:00,0 days 00:30:00,39.0,29.9910,0.159817,2799.680365,6.232877,87137.534247,299454.3960
17518,1.349,2022-09-28T23:00:00+01:00,2022-09-28T23:30:00+01:00,2022-09-28 22:00:00,22:00:00,2022-09-28 22:30:00,22:30:00,0 days 00:30:00,39.0,52.6110,0.159817,2799.840183,6.232877,87143.767123,299507.0070
17519,1.388,2022-09-28T23:30:00+01:00,2022-09-29T00:00:00+01:00,2022-09-28 22:30:00,22:30:00,2022-09-28 23:00:00,23:00:00,0 days 00:30:00,39.0,54.1320,0.159817,2800.000000,6.232877,87150.000000,299561.1390
17520,1.400,2022-09-29T00:00:00+01:00,2022-09-29T00:30:00+01:00,2022-09-28 23:00:00,23:00:00,2022-09-28 23:30:00,23:30:00,0 days 00:30:00,39.0,54.6000,0.159817,2800.159817,6.232877,87156.232877,299615.7390


In [83]:
clean_data["Initial Return on Investment (p)"] = (initial_outlay*100) - clean_data["Cumulative Cost Saved (p)"]
clean_data

,Consumption (kWh),Start,End,iso_start,start_time,iso_end,end_time,time_delta,Cost (pkWh),30 min cost,Solar Output 30 mins (kWh),Cumulative Solar Output (kWh),Cost Saved 30 mins (p),Cumulative Cost Saved (p),Cumulative Non-Solar Cost (p),Initial Return on Investment (p)
0,0.507,2021-09-29T00:00:00+01:00,2021-09-29T00:30:00+01:00,2021-09-28 23:00:00,23:00:00,2021-09-28 23:30:00,23:30:00,0 days 00:30:00,39.0,19.7730,0.159817,0.159817,6.232877,6.232877,19.7730,629993.767123
1,1.318,2021-09-29T00:30:00+01:00,2021-09-29T01:00:00+01:00,2021-09-28 23:30:00,23:30:00,2021-09-29 00:00:00,00:00:00,0 days 00:30:00,7.5,9.8850,0.159817,0.319635,1.198630,7.431507,29.6580,629992.568493
2,1.779,2021-09-29T01:00:00+01:00,2021-09-29T01:30:00+01:00,2021-09-29 00:00:00,00:00:00,2021-09-29 00:30:00,00:30:00,0 days 00:30:00,7.5,13.3425,0.159817,0.479452,1.198630,8.630137,43.0005,629991.369863
3,1.273,2021-09-29T01:30:00+01:00,2021-09-29T02:00:00+01:00,2021-09-29 00:30:00,00:30:00,2021-09-29 01:00:00,01:00:00,0 days 00:30:00,7.5,9.5475,0.159817,0.639269,1.198630,9.828767,52.5480,629990.171233
4,1.309,2021-09-29T02:00:00+01:00,2021-09-29T02:30:00+01:00,2021-09-29 01:00:00,01:00:00,2021-09-29 01:30:00,01:30:00,0 days 00:30:00,7.5,9.8175,0.159817,0.799087,1.198630,11.027397,62.3655,629988.972603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17517,0.769,2022-09-28T22:30:00+01:00,2022-09-28T23:00:00+01:00,2022-09-28 21:30:00,21:30:00,2022-09-28 22:00:00,22:00:00,0 days 00:30:00,39.0,29.9910,0.159817,2799.680365,6.232877,87137.534247,299454.3960,542862.465753
17518,1.349,2022-09-28T23:00:00+01:00,2022-09-28T23:30:00+01:00,2022-09-28 22:00:00,22:00:00,2022-09-28 22:30:00,22:30:00,0 days 00:30:00,39.0,52.6110,0.159817,2799.840183,6.232877,87143.767123,299507.0070,542856.232877
17519,1.388,2022-09-28T23:30:00+01:00,2022-09-29T00:00:00+01:00,2022-09-28 22:30:00,22:30:00,2022-09-28 23:00:00,23:00:00,0 days 00:30:00,39.0,54.1320,0.159817,2800.000000,6.232877,87150.000000,299561.1390,542850.000000
17520,1.400,2022-09-29T00:00:00+01:00,2022-09-29T00:30:00+01:00,2022-09-28 23:00:00,23:00:00,2022-09-28 23:30:00,23:30:00,0 days 00:30:00,39.0,54.6000,0.159817,2800.159817,6.232877,87156.232877,299615.7390,542843.767123


In [85]:
clean_data.to_csv(r"../data/processed_data.csv")

In [99]:
my_data = pd.DataFrame(
    {
        "person": ["hassan", "brad", "hassan", "hassan", "brad"],
        "invoice amount": [1000, 1000, 200, 50, 2500],
        "date": [
            datetime(2022, 1, 1, 12, 0, 0),
            datetime(2022, 1, 2, 12, 0, 0),
            datetime(2022, 2, 1, 12, 0, 0),
            datetime(2022, 2, 2, 12, 0, 0),
            datetime(2022, 1, 1, 12, 0, 0),
        ]
    }
)
my_data

,person,invoice amount,date
0,hassan,1000,2022-01-01 12:00:00
1,brad,1000,2022-01-02 12:00:00
2,hassan,200,2022-02-01 12:00:00
3,hassan,50,2022-02-02 12:00:00
4,brad,2500,2022-01-01 12:00:00


In [103]:
my_data["other column"] = 0

data_copy = my_data.loc[
    (my_data["person"] == "hassan")
    & (my_data["date"] < datetime(2022, 2, 2)),
].copy()

data_copy

,person,invoice amount,date,other column
0,hassan,1000,2022-01-01 12:00:00,0
2,hassan,200,2022-02-01 12:00:00,0
